### Parsiranje nastavnog plana

#### Ulazni fajl:
- ```Plan99.xls```

#### Struktura ulaznog fajla
- Informacije o svim **studijskim programima**, jedan studijski program je jedan sheet
- Za svaki studijski program, informacije o svim **predmetima** koji se održavaju u određenom semestru (zimski ili letnji)
- Za svaki predmet, informacije o **broju semestra, stepenu studija, šifri predmeta i fondu časova** (predavanja i vežbe)

### Dostupni sheet-ovi ulaznog fajla

![Plan sheet-ovi](../docs/plan_sheetovi.png)

### Primer sheet-a ulaznog fajla

![Primer plana](../docs/plan_primer.png)

In [ ]:
import pandas as pd
import uuid
from model_parser import *

### Utility funkcije

In [ ]:
import re
from parser_utils import roman_to_arab, semestar_to_godina, semestar_to_oznaka, cyrilic_to_latin, sheet_name_to_raspored_name

In [ ]:
def extract_plan_num(plan):
    # find first numeric value in plan
    plan_num = int(re.findall(r'\d+', plan)[0])
    return plan_num

#### Parsiranje studijskih programi

- pronađeno **35** studijskih programa (rasporeda)
- ukupno **184** oznake studijskih programa (sa grananjem i podsmerovima)
- oznake koje nisu pronađene u realizaciji, ali postoje u planu:
    - IM8, RUS, SP0, SL0

In [ ]:
def parse_studijski_programi(
        file_path: str = '../data/Plan99.xls'
) -> list[StudijskiProgram]:
    # read all sheets
    plan = pd.read_excel(file_path, sheet_name=None)

    # drop unused sheets
    plan.pop('Podaci')
    plan.pop('MAS (2)')

    # name mappings
    column_names = ['Struka', 'Nivo', 'Stepen', 'Oznaka SS']
    attr_names = ['naziv', 'nivo', 'stepen', 'oznaka']
    name_mapping = {key: value for (key, value) in zip(column_names, attr_names)}
    
    studijski_programi = []
    for _, sheet in plan.items():
        # preprocess
        sheet = sheet[column_names]
        sheet = sheet.rename(columns=name_mapping)
        sheet = sheet.drop_duplicates()
        sheet = sheet.dropna()
        sheet = sheet.astype({'stepen': 'int32', 'nivo': 'int32'})
        # create objects
        for _, row in sheet.iterrows():
            oznaka = cyrilic_to_latin(row.oznaka)
            naziv = cyrilic_to_latin(row.naziv)
            stud_program = StudijskiProgram(str(uuid.uuid4()), row.stepen, row.nivo, oznaka, naziv)
            studijski_programi.append(stud_program)
    
    return studijski_programi

#### Parsiranje predmeta
- pronađeno **1627** predmeta

In [ ]:
def parse_predmeti(
        studijski_programi: list[StudijskiProgram],
        file_path: str = '../data/Plan99.xls'
) -> list[Predmet]:
    # read all sheets
    plan = pd.read_excel(file_path, sheet_name=None)

    # drop unused sheets
    plan.pop('Podaci')
    plan.pop('MAS (2)')

    # name mappings
    column_names = ['Nastavni plan', 'Šifra Struke', 'Katedra', 'Semestar', 'Šifra predmeta', 'Naziv predmeta', 'Fond pred.', 'Fond aud. v.', 'Nivo', 'Stepen', 'Oznaka SS']
    attr_names = ['plan', 'sifra_struke', 'katedra', 'semestar', 'sifra', 'naziv', 'fond_pred', 'fond_vezbe', 'nivo', 'stepen', 'oznaka_sp']
    name_mapping = {key: value for (key, value) in zip(column_names, attr_names)}
    
    predmeti  = []
    for _, sheet in plan.items():
        # preprocess
        sheet = sheet[column_names]
        sheet = sheet.rename(columns=name_mapping)
        sheet = sheet.drop_duplicates()
        sheet = sheet.dropna()
        for _, row in sheet.iterrows():
            # find studijski program that predmet belongs to
            stud_program = next((x for x in studijski_programi if x.stepen == row.stepen and x.nivo == row.nivo and x.oznaka == row.oznaka_sp), None)
            sifra = cyrilic_to_latin(row.sifra)
            naziv = cyrilic_to_latin(row.naziv)
            semestar_arapski = roman_to_arab(row.semestar)
            godina = semestar_to_godina(semestar_arapski)
            semestar = semestar_to_oznaka(semestar_arapski)
            plan = extract_plan_num(row.plan)
            predmet = Predmet(str(uuid.uuid4()), sifra, plan, naziv, godina, semestar, row.fond_pred, stud_program.id, row.fond_vezbe, row.sifra_struke)
            predmeti.append(predmet)
    return predmeti

### Izmena imena studijskih programa

- tačna imena se nalaze u ```Realizacija```
- netačna imena u ```Plan```

In [ ]:
def rename_studijski_programi(
        stud_programi_plan: list[StudijskiProgram],
        stud_programi_realizacija: list[StudijskiProgram]
) -> list[StudijskiProgram]:
    for stud_program_plan in stud_programi_plan:
        stud_program = next((x for x in stud_programi_realizacija \
                            if x.oznaka == stud_program_plan.oznaka and \
                                x.stepen == stud_program_plan.stepen and \
                                x.nivo == stud_program_plan.nivo), None)
        if stud_program:
            stud_program_plan.naziv = stud_program.naziv
    return stud_programi_plan

### Parsiranje informacija o kombinovanju rasporeda

- Kreiranje struktura za kombinovanje više studijskih programa u jedan "štampani" raspored
- E2 se deli na više studijskih programa posle 2. godine, ali se svi nalaze u jednom rasporedu
- ```Plan99.xls``` ima spisak studijskih programa koji pripadaju istom rasporedu
    - 1 sheet = 1 raspored, svi studijski programi u sheet-u se "štampaju" na istom rasporedu

```Struktura:
RasporedPrikaz
{
  "nazivRasporeda": "Racunarstvo i automatika", -> uzima se za ime fajla
  "studProgrami": [studProgram_id, studProgram_id, studProgram_id]
}
```

In [ ]:
# DOPUNE
# CRUD
# dodavanje novog rasporeda => dodavanje novog entiteta
# dodavanje novog studijskog programa u raspored => dodavanje id-a u listu
# studijski programi postoje u sistemu => izmena rasporeda je CRUD (samo (od)linkovanje novog entiteta)

In [ ]:
def create_raspored_spajanje(
        studijski_programi: list[StudijskiProgram],
        file_path: str = '../data/Plan99.xls'
):
    # read all sheets
    plan = pd.read_excel(file_path, sheet_name=None)

    # drop unused sheets
    plan.pop('Podaci')
    plan.pop('MAS (2)')


    # ne postoji pojedinacan raspored na sajtu na koji se mapiraju
    plan.pop('OSSET')
    plan.pop('OSSEET')

    # name mappings
    column_names = ['Nivo', 'Stepen', 'Oznaka SS']
    attr_names = ['nivo', 'stepen', 'oznaka_sp']
    name_mapping = {key: value for (key, value) in zip(column_names, attr_names)}
    
    rasporedi = []
    for sheet_name, sheet in plan.items():
        # preprocess
        sheet = sheet[column_names]
        sheet = sheet.rename(columns=name_mapping)
        sheet = sheet.drop_duplicates()
        sheet = sheet.dropna()
        stud_programi_for_raspored = []
        for _, row in sheet.iterrows():
            # find studijski program with given oznaka
            stud_program_id = next((x.id for x in studijski_programi if x.stepen == row.stepen and x.nivo == row.nivo and x.oznaka == row.oznaka_sp), None)
            stud_programi_for_raspored.append(stud_program_id)
        rasporedi.append(RasporedPrikaz(sheet_name_to_raspored_name(sheet_name), stud_programi_for_raspored))
    return rasporedi

### Ponovno generisanje Studentskih grupa sa novim id-evima studijskih programa

- studijski programi dolaze iz ```Plan```
- studentske grupe imaju id-eve iz ```Realizacija```
- ponovno generisanje kako bi bili dobri id-evi

#### (E) Ekstrakcija podataka o **studentskim grupama**

##### Primer ulaznih podataka
- Studentske grupe - trenutni broj
    - Excel
    - postoji header red
    - ```data/Grupe_Stud_sluzba_2022Z_OSP_proba.xlsx```
    - raznovrsni zapisi broja studenata po grupi, podržani sledeći:
        - 15
        - gr 15 sa 12st
        - Gr. 1 po 32 st.
        - po 15st., Maket. gr.1,2,3,4 i Principi.  Gr.5,6,7,8
        - Grupe 31,32  po 15 studenata
        - gr 1,2,3,4 po 16st i gr 5,6 po 8st
        - Gr. od 1 do 5  po 14 st.,
        - Grupe 11- 8 st
        - Grupe 21 do 28 po 11 st.

![Studentske grupe](../docs/studentske_grupe.png)


##### Pomoćne funkcije za transformaciju podataka

- Transformacija rimskih u arapske brojeve
- Transformacija broja semestra (6) u godinu (3) i oznaku (L/Z) 
- Transformacija broja studenata po grupi

In [ ]:
import re
from parser_utils import roman_to_arab, semestar_to_godina, semestar_to_oznaka

In [ ]:
def broj_stud_po_gr(text: str) -> int|str:
    if type(text) == int:
        return text
    # ex. "14 st"
    br = re.findall(r"(\d+)\s?st", text)
    if len(br) == 1:
        return int(br[0])
    # ex. "po 15st., Maket. gr.1,2,3,4 i Principi.  Gr.5,6,7,8"
    grupe_broj = []
    grupe = text.split('i')
    for i in range(len(br)):
        br_grupa = len(grupe[i].split(','))
        grupe_broj.append(str(br_grupa) + ' po ' + br[i])
    # return format: broj_grupa,broj_studenata|broj_grupa,broj_studenata
    return ('|').join(grupe_broj)

In [ ]:
def extract_studentske_grupe(
        studijski_programi_list: list[StudijskiProgram],
        file_path: str = '../data/Grupe_Stud_sluzba_2022Z_OSP_proba.xlsx',
        sheet_name: str = 'Trenutno - letnji'
) -> list[StudentskaGrupa]:
    # relevant columns
    columns = 'C:I'
    column_names = ['semestar_rimski', 'stepen', 'nivo', 'oznaka_sp', 'br_stud', 'br_gr', 'br_stud_po_gr_str']

    stud_grupe = pd.read_excel(file_path, sheet_name=sheet_name, names=column_names, usecols=columns)

    # transform broj semsestra
    stud_grupe['semestar_arapski'] = stud_grupe.apply(lambda row: roman_to_arab(row.semestar_rimski), axis=1)
    stud_grupe['godina'] = stud_grupe.apply(lambda row: semestar_to_godina(row.semestar_arapski), axis=1)
    stud_grupe['semestar'] = stud_grupe.apply(lambda row: semestar_to_oznaka(row.semestar_arapski), axis=1)
    stud_grupe.drop(['semestar_rimski', 'semestar_arapski'], axis=1, inplace=True)
    # transform broj studenata po grupi
    stud_grupe['br_stud_po_gr'] = stud_grupe.apply(lambda row: broj_stud_po_gr(row.br_stud_po_gr_str), axis=1)

    studentska_grupa_list = []
    for _, row in stud_grupe.iterrows():
        stud_prog = next((x for x in studijski_programi_list if x.stepen == row.stepen and x.nivo == row.nivo and x.oznaka == row.oznaka_sp), None)

        if row.br_gr == 0 or row.br_stud == 0:
            continue
        if stud_prog is not None:
            if type(row.br_stud_po_gr) != int:
                # ex. "4 po 16|2 po 8"
                grupe = row.br_stud_po_gr.split('|')
                j = 0
                for grupa in grupe:
                    broj_studenata_grupa = grupa.split(' po ')
                    broj_grupa = int(broj_studenata_grupa[0])
                    broj_studenata_po_grupi = int(broj_studenata_grupa[1])
                    for _ in range(broj_grupa):
                        studentska_grupa_list.append(StudentskaGrupa(str(uuid.uuid4()), j+1, row.godina, row.semestar, broj_studenata_po_grupi, stud_prog.id))
                        j += 1
            else:
                for i in range(row.br_gr):
                    studentska_grupa_list.append(StudentskaGrupa(str(uuid.uuid4()), i+1, row.godina, row.semestar, row.br_stud_po_gr, stud_prog.id))
        else:
            print('nepoznati studijski programi')
            print(row.oznaka_sp)
    return studentska_grupa_list

#### Izvršavanje

In [ ]:
studijski_programi = parse_studijski_programi()
predmeti = parse_predmeti(studijski_programi)

schedule = MeetingSchedule.read_entity_from_file('1_svi_fajlovi')

studijski_programi = rename_studijski_programi(studijski_programi, schedule.studProgramList)
studentske_grupe = extract_studentske_grupe(studijski_programi)

schedule.studProgramList = studijski_programi
schedule.predmetList = predmeti
schedule.studentskaGrupaList = studentske_grupe
ReadWrite.write_to_file(schedule, '2_svi_fajlovi_plan')

rasporedi = create_raspored_spajanje(schedule.studProgramList)
ReadWrite.write_to_file(rasporedi, '2_rasporedi_spajanje_plan')

# mapiranje novih predmeta
print(studijski_programi[0])
print(predmeti[0])
print(studentske_grupe[0])
print(len(studijski_programi))
print(len(predmeti))
print(len(studentske_grupe))

In [ ]:
# Ljubisa: nepronadjeni studijski programi koji se nalaze u planu, a ne nalaze se u realizaciji